In [ ]:
# Importamos las librerías necesarias
import requests
import pandas as pd
import sqlite3
import os

# Conexión a la API
url = 'https://api.openbrewerydb.org/v1/breweries'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    raise Exception(f"Error al obtener datos de la API: {response.status_code}")

# Convertimos los datos a un DataFrame
df = pd.DataFrame(data)

# Crear la conexión a la base de datos SQLite
conn = sqlite3.connect('brewery.db')
cursor = conn.cursor()

# Creamos la tabla 'breweries' con las columnas especificadas
cursor.execute('''
    CREATE TABLE IF NOT EXISTS breweries (
        id TEXT PRIMARY KEY,
        name TEXT,
        brewery_type TEXT,
        address_1 TEXT,
        address_2 TEXT,
        address_3 TEXT,
        city TEXT,
        state_province TEXT,
        postal_code TEXT,
        country TEXT,
        longitude REAL,
        latitude REAL,
        phone TEXT,
        website_url TEXT,
        state TEXT,
        street TEXT
    )
''')

# Insertamos los datos en la tabla 'breweries'
df.to_sql('breweries', conn, if_exists='replace', index=False)

# Verificar si los datos se guardaron correctamente
stored_data = pd.read_sql('SELECT * FROM breweries', conn)

# Comparar los datos originales con los almacenados (para la auditoría)
diferencias = pd.concat([df, stored_data]).drop_duplicates(keep=False)

# Guardar el archivo de auditoría con cantidad de registros
with open('auditoria.txt', 'w') as file:
    file.write(f"Cantidad de registros en el DataFrame original: {len(df)}\n")
    file.write(f"Cantidad de registros en la base de datos SQLite: {len(stored_data)}\n")
    
    if not diferencias.empty:
        file.write("\nDiferencias entre los datos originales y los almacenados:\n")
        file.write(diferencias.to_string())
    else:
        file.write("\nNo hay diferencias en los datos.\n")

# Guardar los datos en formato CSV
df.to_csv('breweries_data.csv', index=False)

# Guardar los datos en formato Excel
df.to_excel('breweries_data.xlsx', index=False)

# Cerrar la conexión a la base de datos
conn.close()

# Mostrar mensajes de éxito
print("Datos almacenados en la base de datos y archivos generados exitosamente.")
print("Archivo de auditoría creado exitosamente.")
